In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/goodbooks10k-bookscsv/books.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
df[df['ratings_count'] == df['ratings_1'] + df['ratings_2'] + df['ratings_3'] + df['ratings_4'] + df['ratings_5']].shape

In [ ]:
df[df['average_rating'] == (df['ratings_1'] + 2*df['ratings_2'] + 3*df['ratings_3'] + 4*df['ratings_4'] + 5*df['ratings_5'])/df['ratings_count']].shape

In [ ]:
df['new_col'] = df['ratings_1'] + df['ratings_2'] + df['ratings_3'] + df['ratings_4'] + df['ratings_5']
df[df['average_rating'] == (df['ratings_1'] + 2*df['ratings_2'] + 3*df['ratings_3'] + 4*df['ratings_4'] + 5*df['ratings_5'])/df['new_col']].shape

In [ ]:
# prepossing
# takeing columns isbn13, authors, original_publication_year, original_title, image_url
# make a new column rating with ratings_1, ratings_2, ratings_3, ratings_4, ratings_5

# formating
# change name of original_publication_year to year & original_title to title
# change dtype of isbn13 from float64 to int64 & original_publication_year from float64 to int64
# make a new column tags with original_title, authors
# make a new column similar_books with cosine_similarity

In [ ]:
# prepossinng
df = df.iloc[:,[6,7,8,9,16,17,18,19,20,21]]
df['rating'] = round(((df['ratings_1'] + 2*df['ratings_2'] + 3*df['ratings_3'] + 4*df['ratings_4'] + 5*df['ratings_5'])/(df['ratings_1'] + df['ratings_2'] + df['ratings_3'] + df['ratings_4'] + df['ratings_5'])),1)
df.drop(columns=['ratings_1','ratings_2','ratings_3','ratings_4','ratings_5'],inplace=True)

In [ ]:
df.info()

In [ ]:
def collapse(L):
    L1 = ""
    for i in L:
        L1 = L1 + (i.replace(" ",""))
    return L1

In [ ]:
# formating
df.rename(columns={'original_publication_year': 'year','original_title':'title'},inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df = df.astype({"isbn13": int, "year": int})
df['tags'] = df['authors'].apply(collapse) + ", " + df['title']
df.head()

In [ ]:
x = df['tags'].values.tolist()
y = ''
for i in x:
    y = y + i
    
len(set(y.split()))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [ ]:
def recommend(books):
    L = []
    index = df[df['title'] == books].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key= lambda x: x[1])
    for i in distances[1:11]:
        L.append(df.iloc[i[0]].isbn13)
    
    distances.clear()
    return L

In [ ]:
df['similar_books'] = df['title'].apply(recommend)

In [ ]:
df.head()

In [ ]:
books = df.drop(columns='tags')
books.shape

In [ ]:
import pickle
pickle.dump(books,open('books.pkl','wb'))